# imports

In [4]:
%run read_matlab_file.ipynb
%run LC_calibration.ipynb

read_matlab_file.ipynb loaded successfully!
read_matlab_file.ipynb loaded successfully!
read_matlab_file.ipynb loaded successfully!
convert_erg.ipynb loaded successfully!
read_matlab_file.ipynb loaded successfully!
read_matlab_file.ipynb loaded successfully!
convert_erg.ipynb loaded successfully!
safety_time.ipynb loaded successfully!
Absolute time correction subtracted from tus array and T0
187 valleys removed
erg converted to keV
df sorted by adr and tus
safety time events flagged. Found 287 flags
(24587, 11)
LC_calibration loaded successfully!


# XPEC data preperation

### TODO
    - Extract the bins from the txt file
    - Implement function for T0 & tus correction (ToF). COMMENT: still 300ms error after calibrating GRB190206
    - Implement function for cutting the tus & keV array at time interval given by Anastasia or GCN report 
    - Bin the keV into the predefined bins
    
    -Do all the steps above for GRB200415A and cross check results

Absolute time correction subtracted from tus array and T0
187 valleys removed
erg converted to keV
df sorted by adr and tus
safety time events flagged. Found 287 flags


In [12]:
#time of flight from KW to ASIM. Given in seconds
ToF = {"GRB180720":-4.2377,"GRB181222":2.8716, "GRB190206":4.9225, "GRB190305":3.4460,
      "GRB190606":4.0316,"GRB200415":4.1840,"GRB200521":-0.6226,"GRB200716":3.0155,
      "GRB201227":0.7142, "GRB210424":-0.5033, "GRB210619":-2.6183}

In [12]:
def ToF_correction(grb_id=None):
    ToF_us = ToF[grb_id]*1e6
    df,T0 = complete_df(grb_id=grb_id)
    df["tus"], T0_calibrated = df["tus"] - ToF_us, T0 - timedelta(microseconds=ToF_us) #calibrating 
    return df,T0,T0_calibrated

In [6]:
df,T0 = complete_df(grb_id="GRB190206")

Absolute time correction subtracted from tus array and T0
187 valleys removed
erg converted to keV
df sorted by adr and tus
safety time events flagged. Found 287 flags


In [9]:
print(T0)
T1 = T0 - timedelta(microseconds=4.9225*1e6)
print(T1)

2019-02-06 03:49:28.525262
2019-02-06 03:49:23.602762


In [10]:
print(T1)

AttributeError: 'datetime.datetime' object has no attribute 'seconds'